# 1. 完整的模型训练套路(一)

In [ ]:
# 准备数据集
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter

from model import *
from torch.utils.data import DataLoader

train_data = torchvision.datasets.CIFAR10(root="../../dataset",train=True,transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10(root="../../dataset",train=False,transform=torchvision.transforms.ToTensor(),
                                          download=True)

# 查看数据集大小 length 长度
train_data_size = len(train_data)
test_data_size  = len(test_data)
print("训练数据集长度为:{}",format(train_data_size))
print("测试数据集长度为:{}",format(test_data_size))

# 利用 DataLoader 来加载数据集
train_dataloader = DataLoader(train_data,batch_size=64)
test_dataloader = DataLoader(test_data,batch_size=64)

# ###### 打印所有标签
# labels = set(train_data.targets)
# print("所有标签列表:", labels)

###### 打印标签与类别名称的对应关系
## <class 'list'> ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
print(type(train_data.classes), train_data.classes)

for index, class_name in enumerate(train_data.classes):
    print(f"标签 {index}: {class_name}")
    

# ###### 从 DataLoader 中获取一条数据
# for images, labels in train_dataloader:
#     sample_image = images[0]  # 获取第一张图片
#     sample_label = labels[0]   # 获取对应的标签
    
#     print("样本图像形状:", sample_image.shape)  # 打印图像形状
#     print("样本标签:", sample_label.item())    # 打印标签
#     break  # 只需查看第一批数据，使用 break 跳出循环


# 创建网络模型
tudui = Tudui()

# 定义损失函数 用交叉熵
loss_fn = nn.CrossEntropyLoss()

# 定义优化器 选择的是随机梯度下降
# 1e-2 = 1*(10)^(-2) = 1/100 =0.01
learning_rate = 1e-2
optimizer = torch.optim.SGD(tudui.parameters(),lr=learning_rate)

# 设置训练网络的一些参数
# 记录训练的次数
total_train_step = 0
# 记录测试的次数
total_test_step = 0
# 训练的轮数
epoch = 10

Files already downloaded and verified
Files already downloaded and verified
训练数据集长度为:{} 50000
测试数据集长度为:{} 10000
<class 'list'> ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
标签 0: airplane
标签 1: automobile
标签 2: bird
标签 3: cat
标签 4: deer
标签 5: dog
标签 6: frog
标签 7: horse
标签 8: ship
标签 9: truck


In [ ]:
# 添加tensorboard
writer = SummaryWriter("../net_logs")

for i in range(epoch):
    print("-----------第{}轮训练开始-----------".format(i+1))

    # -- 训练步骤开始
    for data in train_dataloader:
        imgs,targets = data
        outputs = tudui(imgs)
        loss = loss_fn(outputs,targets)

        # 优化器优化模型
        # 利用优化器将梯度清零
        optimizer.zero_grad()
        # 利用反向传播得到每个梯度的结点
        loss.backward()
        # 调用优化器
        optimizer.step()

        # 训练次数+1
        total_train_step += 1
        if total_train_step % 100 == 0:
            print("训练次数:{},Loss:{}".format(total_train_step,loss))
            writer.add_scalar("train_loss",loss.item(), total_train_step)

    # -- 测试步骤开始
    # 计算整个数据集上的loss
    total_test_loss = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets = data
            outputs = tudui(imgs)
            # 计算数据的损失
            loss = loss_fn(outputs,targets)
            # 计算整体损失
            total_test_loss = total_test_loss + loss
    print("整体测试集上的loss:{}".format(total_test_loss))
    writer.add_scalar("test_loss",total_test_loss, total_test_step)
    total_test_step += 1
    print(f"-----------第{total_test_step}轮测试结束-----------")
    
    

    # 保存训练时每一轮的模型
    torch.save(tudui, "./model_pth/tudui_{}.pth".format(i))
    print('模型已保存')

writer.close()

-----------第1轮训练开始-----------
训练次数:100,Loss:2.295762062072754
训练次数:200,Loss:2.290508270263672
训练次数:300,Loss:2.2823140621185303
训练次数:400,Loss:2.2400593757629395
训练次数:500,Loss:2.159593105316162
训练次数:600,Loss:2.060598134994507
训练次数:700,Loss:2.0133888721466064
整体测试集上的loss:317.5434265136719
------------第0轮测试结束------------
模型已保存
-----------第2轮训练开始-----------
训练次数:800,Loss:1.9282879829406738
训练次数:900,Loss:1.8913767337799072
训练次数:1000,Loss:1.9943794012069702
训练次数:1100,Loss:1.9976814985275269
训练次数:1200,Loss:1.7579548358917236
训练次数:1300,Loss:1.7011998891830444
训练次数:1400,Loss:1.7928193807601929
训练次数:1500,Loss:1.8460185527801514
整体测试集上的loss:311.77264404296875
------------第1轮测试结束------------
模型已保存
-----------第3轮训练开始-----------
训练次数:1600,Loss:1.813776969909668
训练次数:1700,Loss:1.65890634059906
训练次数:1800,Loss:1.9117518663406372
训练次数:1900,Loss:1.7380167245864868
训练次数:2000,Loss:1.9327287673950195
训练次数:2100,Loss:1.520822286605835
训练次数:2200,Loss:1.474498987197876
训练次数:2300,Loss:1.7858017683029175
整体测试集上的lo

# 2. 完整的模型训练套路(二)
* 增加优化代码：添加测试数据集正确率
* 正确率的计算思路

假设有一个批次包含 3 个样本，输出和目标如下：<br>

输出 (outputs):

[[2.0, 1.0, 0.1, 0.2, 0.5, 0.0, 0.3, 0.1, 0.1, 0.4],  # 真实标签为 0 

 [0.1, 0.4, 2.0, 0.3, 0.1, 0.0, 0.1, 0.2, 0.1, 0.2],  # 真实标签为 2 

 [0.3, 0.5, 0.1, 1.0, 0.2, 0.0, 0.1, 0.1, 2.0, 0.3]]  # 真实标签为 8 


目标 (targets): 

[0, 2, 8]       
<br>
在这个例子中，损失函数会：<br>
对每行应用 Softmax，计算每个样本的类别概率。<br>
取每个样本对应真实标签的概率，应用交叉熵公式计算损失。<br>
对所有样本的损失取平均，得到最终的损失值。

In [10]:
import torch
outputs = torch.tensor([[0.1, 0.2],
                        [0.5, 0.4]])
# 0的时候按列比较，1的时候按行比较
print(outputs.argmax(0))
print(outputs.argmax(1))

tensor([1, 1])
tensor([1, 0])


In [16]:
# 添加tensorboard
writer = SummaryWriter("../net_logs")

for i in range(epoch):
    print("-----------第{}轮训练开始-----------".format(i+1))

    # -- 训练步骤开始
    for data in train_dataloader:
        imgs,targets = data
        outputs = tudui(imgs)
        loss = loss_fn(outputs,targets)

        # 优化器优化模型
        # 利用优化器将梯度清零
        optimizer.zero_grad()
        # 利用反向传播得到每个梯度的结点
        loss.backward()
        # 调用优化器
        optimizer.step()

        # 训练次数+1
        total_train_step += 1
        if total_train_step % 100 == 0:
            print("训练次数:{},Loss:{}".format(total_train_step,loss))
            writer.add_scalar("train_loss",loss.item(), total_train_step)

    # -- 测试步骤开始
    # 计算整个数据集上的loss
    total_test_loss = 0
    # 计算整体的正确率 整体正确的个数 初始为0
    total_true_num = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets = data
            outputs = tudui(imgs)
            # 计算数据的损失
            loss = loss_fn(outputs,targets)
            # 计算整体损失
            total_test_loss = total_test_loss + loss

            # 计算准确率
            batch_true_num = (outputs.argmax(1)==targets).sum()
            total_true_num += batch_true_num         
    test_accuracy = total_true_num/test_data_size

    print("整体测试集上的loss:{}".format(total_test_loss))
    print(f"整体测试集上的正确率:{test_accuracy}")

    writer.add_scalar("test_2_loss",total_test_loss, total_test_step)
    writer.add_scalar("test_2_accuracy",test_accuracy,total_test_step)

    total_test_step += 1
    print(f"-----------第{total_test_step}轮测试结束-----------")    


    # 保存训练时每一轮的模型
    torch.save(tudui, "./model_pth/tudui_2_{}.pth".format(i))
    print('模型已保存')

writer.close()

-----------第1轮训练开始-----------
训练次数:100,Loss:2.2818009853363037
训练次数:200,Loss:2.272014856338501
训练次数:300,Loss:2.1814661026000977
训练次数:400,Loss:2.12896990776062
训练次数:500,Loss:1.9971290826797485
训练次数:600,Loss:2.010012626647949
训练次数:700,Loss:1.9645278453826904
整体测试集上的loss:307.31292724609375
整体测试集上的正确率:0.2976999878883362
-----------第1轮测试结束-----------
模型已保存
-----------第2轮训练开始-----------
训练次数:800,Loss:1.8099256753921509
训练次数:900,Loss:1.7601419687271118
训练次数:1000,Loss:1.9045765399932861
训练次数:1100,Loss:1.9873415231704712
训练次数:1200,Loss:1.637665033340454
训练次数:1300,Loss:1.592820405960083
训练次数:1400,Loss:1.6909105777740479
训练次数:1500,Loss:1.7544349431991577
整体测试集上的loss:292.11785888671875
整体测试集上的正确率:0.3384000062942505
-----------第2轮测试结束-----------
模型已保存
-----------第3轮训练开始-----------
训练次数:1600,Loss:1.6719130277633667
训练次数:1700,Loss:1.625885248184204
训练次数:1800,Loss:1.9291118383407593
训练次数:1900,Loss:1.6375724077224731
训练次数:2000,Loss:1.9054876565933228
训练次数:2100,Loss:1.5076005458831787
训练次数:2200,Loss:1.4

# 29. 完整的模型训练套路(三)
浅浅修改(tao就不写了), 增加了

tudui.train() # 作用:将网络设置成训练模式 只对部分网络层有作用,例如Dropout层,BatchNorm层等

tudui.eval() # 设置模型进入验证状态 只对特定层有作用,例如Dropout层,BatchNorm层等